In [2]:
from pyspark.sql import SparkSession

# Cerrar sesión actual
try:
    spark.stop()
    print("✅ Sesión anterior cerrada")
except:
    print("ℹ️ No había sesión previa")

# Reiniciar con configuración explícita del JAR
spark = (SparkSession.builder
    .appName("Validacion-partition")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "1")
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "1g")
    .config("spark.jars", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.driver.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.executor.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .enableHiveSupport()
    .getOrCreate())

print("✅ Spark version:", spark.version)
print("✅ Master URL:", spark.sparkContext.master)
print("✅ Spark reiniciado con driver JDBC")

25/09/12 02:54:23 WARN Dispatcher: Message RemoteProcessDisconnected(172.18.0.11:45894) dropped. Could not find BlockManagerEndpoint1.


✅ Sesión anterior cerrada
✅ Spark version: 2.4.5
✅ Master URL: spark://spark-master:7077
✅ Spark reiniciado con driver JDBC


In [3]:
# Crear un DataFrame con 1 millón de registros
df = spark.range(0, 1_000_000)
print("Número de particiones iniciales:", df.rdd.getNumPartitions())


Número de particiones iniciales: 2


In [4]:
# Aumentar particiones (shuffle)
df_repart = df.repartition(20)
print("Particiones después de repartition(20):", df_repart.rdd.getNumPartitions())


Particiones después de repartition(20): 20


In [17]:
data_hdfs = "hdfs://namenode:8020/bronze/practica/"

In [22]:
df_repart.write.mode("overwrite").parquet(data_hdfs)


In [23]:
df=df_repart

In [24]:
df.show()

+------+
|    id|
+------+
|956256|
|660444|
|683865|
|510992|
|964350|
|590703|
|623570|
|651757|
|562552|
|585251|
|743592|
|836189|
|963551|
|680118|
|919581|
|890615|
|950043|
|923748|
|953942|
|503586|
+------+
only showing top 20 rows



In [26]:
df.cache()
df.count()

1000000

In [12]:
df1=df

In [27]:
df.coalesce(2).write.mode("overwrite").parquet(data_hdfs)

In [28]:
df=spark.read.parquet(data_hdfs)
df.show()

+------+
|    id|
+------+
| 29077|
|241354|
|470868|
|266683|
|198432|
|307106|
|247056|
| 94811|
|265530|
|284196|
| 78042|
| 70560|
| 78714|
|210265|
|338262|
|135195|
|267404|
|299700|
|355065|
|121636|
+------+
only showing top 20 rows



In [29]:
spark.catalog.clearCache()

In [30]:
spark.stop()